<h1 align=center><font size = 10>The "Battle" of Neighborhoods : Manhattan vs Staten Island </font></h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

Task 1. <a href="#item1">Introduction</a>

Task 2. <a href="#item2">Data</a>

Task 3. <a href="#item3">Methodology</a>


   
</font>
</div>



# Introduction

## <span style="color:red">  KEEP CALM, STAY HOME </span>

The world has faced a new challenge - to resist to new coronavirus COVID 19. 
The virus has penetrated all countries. The greatest threat of contamination to the inhabitants of megacities.


While scientists are looking for a vaccine, and doctors are fighting for people 's lives, our task is  **KEEP CALM, STAY HOME** .

New York city is heroic against the virus. Unfortunately, it 's not stopped yet. You can find daily updates of information about people who have tested positive for COVID-19 in NYC : https://www1.nyc.gov/site/doh/covid/covid-19-data.page

Due to the coronavirus outbreak, all of NYC’s nonessential businesses have been ordered to keep their workforces at home, and all nonessential gatherings are temporarily banned.  
 

**Residents Manhattan** began to search for housing in other parts of the city. They no longer need restaurants and places of entertainment, they need places for privacy and relaxation.

NYC has 5 Boroughs : Bronx, Brooklyn, Manhattan, Queens and Staten Island. https://www.nycgo.com/neighborhoods-boroughs/about-nyc-five-boroughs/ .
Bronx, Brooklyn and Queens have a high level of diseases. Let's have a look at **Staten Island** . Easily accessible by a scenic ride on the Staten Island Ferry, the greenest borough feels like a getaway within the City. Beyond a charming North Shore rich with maritime history, Staten Island is best known for its beaches, vast parkland and even a fully preserved colonial village.

**Real Estate Agency**( hereinafter referred to as “REA”) would like to compare those two Boroughs (Manhattan vs Staten Island) and  to focus on a list of places of **Basic necessities**: hospitals, pharmacies, food stores, repair workshops, veterinary clinics and so on. 
The full list is provided here :  https://esd.ny.gov/guidance-executive-order-2026?cid=NYCEM_MEM_Coronavirus11_20200320
 
<span style="color:red"> ****Can REA offer to rent housing with all Basic necessities in Staten Island??****</span>



# Data

We will have a look at the list of Neighborhoods of each Boroughs and more focus on  Manhattan and Staten Island. We use a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. Then we will select the places of Basic necessities , will group the neighborhoods into clusters and will visualize them with  Folium library:

- https://cocl.us/new_york_dataset
- https://geo.nyu.edu/catalog/nyu_2451_34572
- Foursquare API to explore neighborhoods in NYC.



# Methodology

## -Let's download all the dependencies that we will need.

In [1]:
pip install geopy

     |████████████████████████████████| 112kB 24.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## - Download and Explore Dataset

All files are downloaded and placed it on the server, so we can simply run a `wget` command and access the data. So let's go ahead and do that.

In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


Next, let's load the data.

In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Let's take a quick look at the data.

In [5]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [6]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [7]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

####  - Tranform the data into a *pandas* dataframe

In [8]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [9]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [10]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

## - Now we have a list of Boroughs with Neighborhood ,  Latitude and Longitude

In [11]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [12]:
neighborhoods.tail()

,Borough,Neighborhood,Latitude,Longitude
301,Manhattan,Hudson Yards,40.756658,-74.000111
302,Queens,Hammels,40.587338,-73.805530
303,Queens,Bayswater,40.611322,-73.765968
304,Queens,Queensbridge,40.756091,-73.945631
305,Staten Island,Fox Hills,40.617311,-74.081740


## - And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [14]:
neighborhoods.shape

(306, 4)

In [15]:
neighborhoods.dtypes

Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [16]:
neighborhoods.values

array([['Bronx', 'Wakefield', 40.89470517661, -73.84720052054902],
       ['Bronx', 'Co-op City', 40.87429419303012, -73.82993910812398],
       ['Bronx', 'Eastchester', 40.887555677350775, -73.82780644716412],
       ...,
       ['Queens', 'Bayswater', 40.611321691283834, -73.76596781445627],
       ['Queens', 'Queensbridge', 40.756091297094706, -73.94563070334091],
       ['Staten Island', 'Fox Hills', 40.61731079252983,
        -74.08173992211962]], dtype=object)

In [17]:
NG=neighborhoods.groupby('Borough').count()[['Neighborhood']]
NG.sort_values(by='Neighborhood', ascending=False)
print('=========================================================================== ')
print('The list of Boroughs of New York City with total number of  Neighborhoods : ')
print('=========================================================================== ')
print(' ')
NG.head()

The list of Boroughs of New York City with total number of  Neighborhoods : 
 


,Neighborhood
Borough,
Bronx,52
Brooklyn,70
Manhattan,40
Queens,81
Staten Island,63


## - Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [18]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### - Create a map of New York with neighborhoods superimposed on top.

In [19]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

# Let's have a look at Manhattan

## Manhattan

Let's slice the original dataframe and create a new dataframe of the Manhattan data. PLease have a look at the list of all Neighborhoods  

In [20]:
Manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
Manhattan_data.head(40)

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


Let's get the geographical coordinates of Manhattan.

In [21]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.

In [22]:
# create map of Manhattan using latitude and longitude values
map_Manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Manhattan_data['Latitude'], Manhattan_data['Longitude'], Manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Manhattan)  
    
map_Manhattan

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [23]:
CLIENT_ID = '0RJKNKVNGQ4TBXPY0DQN4MRJIXZO1YS3W2MSOLGESJT34MBD' # your Foursquare ID
CLIENT_SECRET = 'ZFY50IOKW3W5PBLSL41OW10HTKG3K2HVX1XWIRJTHPRLVFK1' # your Foursquare Secret
VERSION = '20181102'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0RJKNKVNGQ4TBXPY0DQN4MRJIXZO1YS3W2MSOLGESJT34MBD
CLIENT_SECRET:ZFY50IOKW3W5PBLSL41OW10HTKG3K2HVX1XWIRJTHPRLVFK1


#### Let's explore the Midtown neighborhood in our dataframe.

Get the neighborhood's name.

In [24]:
Manhattan_data.loc[15, 'Neighborhood']

'Midtown'

Get the neighborhood's latitude and longitude values.

In [25]:
neighborhood_latitude = Manhattan_data.loc[15, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Manhattan_data.loc[15, 'Longitude'] # neighborhood longitude valum

neighborhood_name = Manhattan_data.loc[15, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Midtown are 40.75469110270623, -73.98166882730304.


#### Now, let's get the top 100 venues that are in Midtown within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [26]:
# type your answer here
radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=0RJKNKVNGQ4TBXPY0DQN4MRJIXZO1YS3W2MSOLGESJT34MBD&client_secret=ZFY50IOKW3W5PBLSL41OW10HTKG3K2HVX1XWIRJTHPRLVFK1&v=20181102&ll=40.75469110270623,-73.98166882730304&radius=500&limit=100'

Send the GET request and examine the resutls

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e98a28ca2e538001b9a932f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Midtown East',
  'headerFullLocation': 'Midtown East, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 186,
  'suggestedBounds': {'ne': {'lat': 40.759191107206235,
    'lng': -73.97573940939031},
   'sw': {'lat': 40.75019109820623, 'lng': -73.98759824521576}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '3fd66200f964a520d7f11ee3',
       'name': 'Bryant Park',
       'location': {'address': 'E 42nd St',
        'crossStreet': 'btwn 5th & 6th Ave',
        'lat': 40.753621,
        'lng': -73.983265,
        'labeledLatLngs': [{'label': 'disp

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [29]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(105)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Bryant Park,Park,40.753621,-73.983265
1,New York Public Library Terrace,Plaza,40.753017,-73.981480
2,Nat Sherman Townhouse,Smoke Shop,40.753283,-73.980358
3,Sophie's Cuban Cuisine,Cuban Restaurant,40.755927,-73.980598
4,sweetgreen,Salad Place,40.754640,-73.983102
5,CAVA,Mediterranean Restaurant,40.754186,-73.981913
6,Joanna Vargas Skin Care,Spa,40.753136,-73.980721
7,Sofitel New York,Hotel,40.755787,-73.981762
8,Gabriel Kreuther,French Restaurant,40.754143,-73.983505
9,Equinox East 43rd Street,Gym,40.754089,-73.979900


And how many venues were returned by Foursquare?

In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Manhattan

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### - Now write the code to run the above function on each neighborhood and create a new dataframe called **Manhattan_venues**.

In [32]:
# type your answer here

Manhattan_venues = getNearbyVenues(names=Manhattan_data['Neighborhood'],
                                   latitudes=Manhattan_data['Latitude'],
                                   longitudes=Manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


#### Let's check the size of the resulting dataframe

In [33]:
Manhattan_venues.shape

(3158, 7)

In [34]:
print(Manhattan_venues.shape)
Manhattan_venues.head()

(3158, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [35]:
print(Manhattan_venues.shape)
Manhattan_venues.tail()

(3158, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3153,Hudson Yards,40.756658,-74.000111,Playboy Club New York,40.760000,-73.996367,Lounge
3154,Hudson Yards,40.756658,-74.000111,Silver Towers Dog Run,40.760854,-73.999765,Dog Run
3155,Hudson Yards,40.756658,-74.000111,Cachet Boutique Hotel,40.759773,-73.996460,Hotel
3156,Hudson Yards,40.756658,-74.000111,NY Waterway 42nd St Bus,40.760050,-74.003379,Bus Station
3157,Hudson Yards,40.756658,-74.000111,Twilight Cruise By Citysightseeing,40.759744,-74.004096,Boat or Ferry


## - Let's check how many venues were returned for each neighborhood

In [36]:
Manhattan_venues.groupby('Neighborhood').count()['Venue']

Neighborhood
Battery Park City       69
Carnegie Hill           88
Central Harlem          45
Chelsea                100
Chinatown              100
Civic Center           100
Clinton                100
East Harlem             39
East Village           100
Financial District     100
Flatiron               100
Gramercy                81
Greenwich Village      100
Hamilton Heights        58
Hudson Yards            62
Inwood                  56
Lenox Hill             100
Lincoln Square          95
Little Italy           100
Lower East Side         50
Manhattan Valley        53
Manhattanville          49
Marble Hill             25
Midtown                100
Midtown South          100
Morningside Heights     38
Murray Hill            100
Noho                   100
Roosevelt Island        26
Soho                   100
Stuyvesant Town         18
Sutton Place           100
Tribeca                 77
Tudor City              77
Turtle Bay             100
Upper East Side         86
Upper West Side

#### Let's find out how many unique categories can be curated from all the returned venues

In [37]:
print('There are {} uniques categories.'.format(len(Manhattan_venues['Venue Category'].unique())))

There are 323 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [38]:
# one hot encoding
Manhattan_onehot = pd.get_dummies(Manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Manhattan_onehot['Neighborhood'] = Manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Manhattan_onehot.columns[-1]] + list(Manhattan_onehot.columns[:-1])
Manhattan_onehot = Manhattan_onehot[fixed_columns]
Manhattan_onehot.head()


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South Ame

In [39]:
Manhattan_onehot.columns = Manhattan_onehot.columns.str.replace("'","")
Manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctors Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Mens Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South Ameri

And let's examine the new dataframe size.

In [40]:
Manhattan_onehot.shape

(3158, 324)

In [41]:
for col in Manhattan_onehot.columns: 
    print(col) 

Neighborhood
Accessories Store
Adult Boutique
Afghan Restaurant
African Restaurant
American Restaurant
Antique Shop
Arcade
Arepa Restaurant
Argentinian Restaurant
Art Gallery
Art Museum
Arts & Crafts Store
Asian Restaurant
Athletics & Sports
Auditorium
Australian Restaurant
Austrian Restaurant
BBQ Joint
Baby Store
Bagel Shop
Bakery
Bank
Bar
Baseball Field
Basketball Court
Beer Bar
Beer Garden
Beer Store
Bike Rental / Bike Share
Bike Shop
Bike Trail
Bistro
Board Shop
Boat or Ferry
Bookstore
Boutique
Boxing Gym
Brazilian Restaurant
Breakfast Spot
Bridal Shop
Bridge
Bubble Tea Shop
Building
Burger Joint
Burrito Place
Bus Station
Bus Stop
Butcher
Cafeteria
Café
Cajun / Creole Restaurant
Cambodian Restaurant
Camera Store
Candy Store
Cantonese Restaurant
Caribbean Restaurant
Caucasian Restaurant
Cheese Shop
Chinese Restaurant
Chocolate Shop
Circus
Climbing Gym
Clothing Store
Club House
Cocktail Bar
Coffee Shop
College Academic Building
College Arts Building
College Bookstore
College Cafeteri

## -Let's select a list of Basic necessities 

In [42]:
Basic_necessities=Manhattan_onehot[['Neighborhood','Baby Store','Bakery','Bank','Boat or Ferry','Bus Station','Bus Stop','Clothing Store','Doctors Office','Drugstore','Dry Cleaner','Electronics Store','Farmers Market','Fish Market','Heliport','Health & Beauty Service','Gas Station','Grocery Store','Health Food Store','Kids Store','Kitchen Supply Store','Laundry Service','Market','Medical Center','Metro Station','Optical Shop','Pet Service','Pet Store','Pharmacy','Physical Therapist','Trail','Train Station','Veterinarian','Waterfront']]
Basic_necessities.head()

,Neighborhood,Baby Store,Bakery,Bank,Boat or Ferry,Bus Station,Bus Stop,Clothing Store,Doctors Office,Drugstore,Dry Cleaner,Electronics Store,Farmers Market,Fish Market,Heliport,Health & Beauty Service,Gas Station,Grocery Store,Health Food Store,Kids Store,Kitchen Supply Store,Laundry Service,Market,Medical Center,Metro Station,Optical Shop,Pet Service,Pet Store,Pharmacy,Physical Therapist,Trail,Train Station,Veterinarian,Waterfront
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


we have a list of 32 places :

In [43]:
Basic_necessities.shape

(3158, 34)

#### - Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [44]:
Manhattan_grouped = Basic_necessities.groupby('Neighborhood').mean().reset_index()
Manhattan_grouped

,Neighborhood,Baby Store,Bakery,Bank,Boat or Ferry,Bus Station,Bus Stop,Clothing Store,Doctors Office,Drugstore,Dry Cleaner,Electronics Store,Farmers Market,Fish Market,Heliport,Health & Beauty Service,Gas Station,Grocery Store,Health Food Store,Kids Store,Kitchen Supply Store,Laundry Service,Market,Medical Center,Metro Station,Optical Shop,Pet Service,Pet Store,Pharmacy,Physical Therapist,Trail,Train Station,Veterinarian,Waterfront
0,Battery Park City,0.00,0.000000,0.000000,0.028986,0.000000,0.000000,0.014493,0.00,0.000000,0.000000,0.014493,0.000000,0.00,0.000000,0.00,0.000000,0.014493,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.014493,0.000000,0.00,0.000000,0.00,0.000000,0.000000
1,Carnegie Hill,0.00,0.034091,0.011364,0.000000,0.000000,0.000000,0.011364,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.022727,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.011364,0.000000,0.00,0.000000,0.00,0.000000,0.000000
2,Central Harlem,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.022222,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
3,Chelsea,0.00,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.01,0.000000,0.01,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.020000,0.00,0.000000,0.000000,0.000000,0.020000,0.000000,0.01,0.000000,0.00,0.000000,0.000000
4,Chinatown,0.00,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.030000,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.000000,0.000000
5,Civic Center,0.01,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
6,Clinton,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
7,East Harlem,0.00,0.102564,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.025641,0.025641,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.025641,0.00,0.000000,0.00,0.000000,0.000000
8,East Village,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
9,Financial District,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.010000,0.010000,0.00,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000


#### Let's confirm the new size

In [45]:
Manhattan_grouped.shape

(40, 34)

In [46]:
for col in Manhattan_grouped.columns: 
    print(col) 

Neighborhood
Baby Store
Bakery
Bank
Boat or Ferry
Bus Station
Bus Stop
Clothing Store
Doctors Office
Drugstore
Dry Cleaner
Electronics Store
Farmers Market
Fish Market
Heliport
Health & Beauty Service
Gas Station
Grocery Store
Health Food Store
Kids Store
Kitchen Supply Store
Laundry Service
Market
Medical Center
Metro Station
Optical Shop
Pet Service
Pet Store
Pharmacy
Physical Therapist
Trail
Train Station
Veterinarian
Waterfront


#### Let's print each neighborhood along with the TOP 10 most common venues

In [47]:
num_top_venues = 10

for hood in Manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Manhattan_grouped[Manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                venue  freq
0       Boat or Ferry  0.03
1       Grocery Store  0.01
2      Clothing Store  0.01
3   Electronics Store  0.01
4           Pet Store  0.01
5               Trail  0.00
6       Train Station  0.00
7  Physical Therapist  0.00
8            Pharmacy  0.00
9          Kids Store  0.00


----Carnegie Hill----
                venue  freq
0              Bakery  0.03
1       Grocery Store  0.02
2      Clothing Store  0.01
3           Pet Store  0.01
4                Bank  0.01
5       Train Station  0.00
6               Trail  0.00
7        Veterinarian  0.00
8  Physical Therapist  0.00
9            Pharmacy  0.00


----Central Harlem----
                venue  freq
0              Market  0.02
1          Baby Store  0.00
2   Health Food Store  0.00
3        Veterinarian  0.00
4       Train Station  0.00
5               Trail  0.00
6  Physical Therapist  0.00
7            Pharmacy  0.00
8           Pet Store  0.00
9         Pet Service  0.00



#### - Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Manhattan_grouped['Neighborhood']

for ind in np.arange(Manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(40)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Boat or Ferry,Grocery Store,Electronics Store,Pet Store,Clothing Store,Bus Station,Bank,Bus Stop,Gas Station,Doctors Office
1,Carnegie Hill,Bakery,Grocery Store,Pet Store,Clothing Store,Bank,Boat or Ferry,Bus Station,Bus Stop,Gas Station,Doctors Office
2,Central Harlem,Market,Waterfront,Drugstore,Heliport,Fish Market,Farmers Market,Electronics Store,Dry Cleaner,Doctors Office,Gas Station
3,Chelsea,Bakery,Market,Pet Store,Grocery Store,Fish Market,Physical Therapist,Electronics Store,Health & Beauty Service,Farmers Market,Dry Cleaner
4,Chinatown,Bakery,Optical Shop,Pharmacy,Doctors Office,Fish Market,Farmers Market,Electronics Store,Dry Cleaner,Drugstore,Waterfront
5,Civic Center,Baby Store,Medical Center,Bakery,Farmers Market,Laundry Service,Clothing Store,Health & Beauty Service,Fish Market,Electronics Store,Dry Cleaner
6,Clinton,Bakery,Health Food Store,Waterfront,Drugstore,Heliport,Fish Market,Farmers Market,Electronics Store,Dry Cleaner,Doctors Office
7,East Harlem,Bakery,Grocery Store,Pharmacy,Gas Station,Drugstore,Heliport,Fish Market,Farmers Market,Electronics Store,Dry Cleaner
8,East Village,Bakery,Farmers Market,Waterfront,Gas Station,Bank,Boat or Ferry,Bus Station,Bus Stop,Clothing Store,Doctors Office
9,Financial District,Grocery Store,Bakery,Doctors Office,Electronics Store,Farmers Market,Gas Station,Bank,Boat or Ferry,Bus Station,Bus Stop


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [50]:
# set number of clusters
kclusters = 5

Manhattan_grouped_clustering = Manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 2, 2, 0, 0, 4, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Manhattan_merged = Manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Manhattan_merged = Manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Bank,Kids Store,Pharmacy,Waterfront,Drugstore,Fish Market,Farmers Market,Electronics Store,Dry Cleaner,Clothing Store
1,Manhattan,Chinatown,40.715618,-73.994279,2,Bakery,Optical Shop,Pharmacy,Doctors Office,Fish Market,Farmers Market,Electronics Store,Dry Cleaner,Drugstore,Waterfront
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Grocery Store,Bakery,Bank,Market,Pet Store,Kids Store,Clothing Store,Fish Market,Farmers Market,Electronics Store
3,Manhattan,Inwood,40.867684,-73.921210,2,Bakery,Grocery Store,Veterinarian,Pharmacy,Pet Store,Bus Station,Farmers Market,Drugstore,Fish Market,Electronics Store
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Bakery,Bank,Waterfront,Gas Station,Boat or Ferry,Bus Station,Bus Stop,Clothing Store,Doctors Office,Drugstore


In [52]:
Manhattan_merged.dtypes

Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster Labels              int32
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

Finally, let's visualize the resulting clusters

In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Manhattan_merged['Latitude'], Manhattan_merged['Longitude'], Manhattan_merged['Neighborhood'], Manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## -Let's have a look more in details on each cluster

In [54]:
Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 0, Manhattan_merged.columns[[1] + list(range(5, Manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Bank,Kids Store,Pharmacy,Waterfront,Drugstore,Fish Market,Farmers Market,Electronics Store,Dry Cleaner,Clothing Store
5,Manhattanville,Grocery Store,Bakery,Bank,Bus Station,Bus Stop,Gas Station,Boat or Ferry,Clothing Store,Doctors Office,Drugstore
6,Central Harlem,Market,Waterfront,Drugstore,Heliport,Fish Market,Farmers Market,Electronics Store,Dry Cleaner,Doctors Office,Gas Station
9,Yorkville,Health & Beauty Service,Bakery,Bank,Pharmacy,Fish Market,Farmers Market,Drugstore,Heliport,Electronics Store,Dry Cleaner
10,Lenox Hill,Bakery,Grocery Store,Bank,Health Food Store,Pet Store,Clothing Store,Kitchen Supply Store,Dry Cleaner,Fish Market,Farmers Market
13,Lincoln Square,Grocery Store,Bakery,Clothing Store,Pharmacy,Pet Store,Electronics Store,Drugstore,Fish Market,Farmers Market,Dry Cleaner
14,Clinton,Bakery,Health Food Store,Waterfront,Drugstore,Heliport,Fish Market,Farmers Market,Electronics Store,Dry Cleaner,Doctors Office
16,Murray Hill,Grocery Store,Bakery,Pharmacy,Pet Service,Doctors Office,Fish Market,Farmers Market,Electronics Store,Dry Cleaner,Drugstore
19,East Village,Bakery,Farmers Market,Waterfront,Gas Station,Bank,Boat or Ferry,Bus Station,Bus Stop,Clothing Store,Doctors Office
25,Manhattan Valley,Grocery Store,Bakery,Clothing Store,Farmers Market,Gas Station,Bank,Boat or Ferry,Bus Station,Bus Stop,Doctors Office


In [55]:
Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 1, Manhattan_merged.columns[[1] + list(range(5, Manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Boat or Ferry,Heliport,Farmers Market,Pet Service,Gas Station,Waterfront,Drugstore,Fish Market,Electronics Store,Dry Cleaner


In [56]:
Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 2, Manhattan_merged.columns[[1] + list(range(5, Manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Bakery,Optical Shop,Pharmacy,Doctors Office,Fish Market,Farmers Market,Electronics Store,Dry Cleaner,Drugstore,Waterfront
2,Washington Heights,Grocery Store,Bakery,Bank,Market,Pet Store,Kids Store,Clothing Store,Fish Market,Farmers Market,Electronics Store
3,Inwood,Bakery,Grocery Store,Veterinarian,Pharmacy,Pet Store,Bus Station,Farmers Market,Drugstore,Fish Market,Electronics Store
4,Hamilton Heights,Bakery,Bank,Waterfront,Gas Station,Boat or Ferry,Bus Station,Bus Stop,Clothing Store,Doctors Office,Drugstore
8,Upper East Side,Bakery,Grocery Store,Pet Store,Electronics Store,Kitchen Supply Store,Optical Shop,Clothing Store,Bus Stop,Bus Station,Health & Beauty Service
12,Upper West Side,Bakery,Grocery Store,Drugstore,Gas Station,Bank,Boat or Ferry,Bus Station,Bus Stop,Clothing Store,Doctors Office
15,Midtown,Clothing Store,Bakery,Grocery Store,Train Station,Pharmacy,Drugstore,Fish Market,Farmers Market,Electronics Store,Dry Cleaner
17,Chelsea,Bakery,Market,Pet Store,Grocery Store,Fish Market,Physical Therapist,Electronics Store,Health & Beauty Service,Farmers Market,Dry Cleaner
18,Greenwich Village,Bakery,Clothing Store,Optical Shop,Drugstore,Heliport,Fish Market,Farmers Market,Electronics Store,Dry Cleaner,Waterfront
20,Lower East Side,Bakery,Clothing Store,Pharmacy,Bus Stop,Drugstore,Heliport,Fish Market,Farmers Market,Electronics Store,Dry Cleaner


In [57]:
Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 3, Manhattan_merged.columns[[1] + list(range(5, Manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Roosevelt Island,Waterfront,Metro Station,Farmers Market,Dry Cleaner,Heliport,Fish Market,Electronics Store,Drugstore,Doctors Office,Gas Station


In [58]:
Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 4, Manhattan_merged.columns[[1] + list(range(5, Manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East Harlem,Bakery,Grocery Store,Pharmacy,Gas Station,Drugstore,Heliport,Fish Market,Farmers Market,Electronics Store,Dry Cleaner
